In [51]:
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedKFold, cross_val_predict
import numpy as np

from xgboost import XGBRegressor

In [73]:
train_x = pd.read_csv("../results/train_x.csv")
train_y = pd.read_csv("../results/train_y.csv")
imdb_info = pd.read_csv("../results/imdb_results.csv")

In [74]:
all_info = train_x.merge(imdb_info, left_on="player_1_name", right_on="Movie", how = "left")

In [75]:
all_info = all_info.merge(imdb_info, left_on="player_2_name", right_on="Movie", suffixes = ("_player1", "_player2"), how="left")

In [76]:
all_info = all_info.drop(columns=["player_1_name", "player_2_name", "Movie_player1", "Movie_player2", "id_player1", "id_player2"])

In [78]:
xgb_model = XGBRegressor(n_estimators=500)

results = cross_val_score(xgb_model, 
                 X = all_info,
                 y = train_y["plus_minus"],
                 scoring="neg_mean_squared_error",
                 cv=RepeatedKFold()
               )
np.sqrt(abs(results)).mean()

19.71948771900352

In [79]:
results = cross_val_predict(xgb_model, 
                 X = all_info,
                 y = train_y["plus_minus"],
                 cv=5
               )

In [80]:
train_y["predicted"] = results

In [81]:
train_y["correct"] = np.sign(train_y["plus_minus"]) * np.sign(train_y["predicted"])

In [82]:
train_y.loc[train_y["correct"] == -1, :].to_csv("../results/mistakes1.1.csv", index = False)

In [83]:
train_y.to_csv("../results/gen_1.1_model.csv", index = False)